# Custom Label's using Amazon Rekognition

***
This notebook provides a walkthrough of [object detection API](https://docs.aws.amazon.com/rekognition/latest/dg/labels.html) in Amazon Rekognition to identify objects.
***

# Initialize SageMaker Notebook Environment

In [8]:
# Initialise Notebook

import boto3
import botocore
from botocore.config import Config
import datetime
from dateutil.tz import tzlocal
import os


In [14]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

assume_role_cache: dict = {}
def assumed_role_session(role_arn: str, base_session: botocore.session.Session = None):
    base_session = base_session or boto3.session.Session()._session
    fetcher = botocore.credentials.AssumeRoleCredentialFetcher(
        client_creator = base_session.create_client,
        source_credentials = base_session.get_credentials(),
        role_arn = role_arn,
        extra_args = {
        #    'RoleSessionName': None # set this if you want something non-default
        }
    )
    stscredentials = botocore.credentials.DeferredRefreshableCredentials(
        method = 'assume-role',
        refresh_using = fetcher.fetch_credentials,
        time_fetcher = lambda: datetime.datetime.now(tzlocal())
    )
    botocore_session = botocore.session.Session()
    botocore_session._credentials = stscredentials
    return boto3.Session(botocore_session = botocore_session)

session = assumed_role_session('arn:aws:iam::744458063302:role/RekognitionCustomLabels')


#mySession = boto3.session.Session()
awsRegion = session.region_name

my_config = Config(
    region_name = 'us-east-1',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

In [15]:
# Init clients
rekognition = session.client('rekognition', config=my_config)
s3 = boto3.client('s3')

In [16]:
# S3 bucket that contains sample images and videos

# We are providing sample images and videos in this bucket so
# you do not have to manually download/upload test images and videos.

projects = rekognition.describe_projects(
    )

print(projects['ProjectDescriptions'])

[{'ProjectArn': 'arn:aws:rekognition:us-east-1:744458063302:project/M2C-GPS304-CustomModel/1636137228257', 'CreationTimestamp': datetime.datetime(2021, 11, 5, 11, 33, 48, 257000, tzinfo=tzlocal()), 'Status': 'CREATED'}, {'ProjectArn': 'arn:aws:rekognition:us-east-1:744458063302:project/project/1632518713009', 'CreationTimestamp': datetime.datetime(2021, 9, 24, 14, 25, 13, 9000, tzinfo=tzlocal()), 'Status': 'CREATED'}]


In [ ]:
# Create temporary directory
# This directory is not needed to call Rekognition APIs.
# We will only use this directory to download images from S3 bucket and draw bounding boxes

!mkdir m1tmp
tempFolder = 'm1tmp/'